In [1]:
import pyspark
import faker

fake = faker.Faker()

data = [(fake.name(), fake.address(), fake.email(), fake.phone_number()) for _ in range(1000)]

In [15]:
spark = (
    pyspark.sql.SparkSession.builder.appName("example")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.iceberg_catalog", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.iceberg_catalog.type", "hadoop")
    .config("spark.sql.catalog.iceberg_catalog.warehouse", "../warehouse")
    .config("spark.defaultCatalog", "iceberg_catalog")
    .getOrCreate()
)

24/10/08 12:13:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/08 12:13:05 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [16]:
spark.sql("create database if not exists iceberg_db")

DataFrame[]

In [17]:
df = spark.createDataFrame(data, ["name", "address", "email", "phone_number"]).coalesce(1)
df.writeTo("iceberg_catalog.iceberg_db.user").using("iceberg").createOrReplace()  

24/10/08 12:13:11 ERROR Utils: Aborting task                        (0 + 1) / 1]
java.io.UncheckedIOException: Failed to create Parquet file
	at org.apache.iceberg.parquet.ParquetWriter.ensureWriterInitialized(ParquetWriter.java:121)
	at org.apache.iceberg.parquet.ParquetWriter.flushRowGroup(ParquetWriter.java:210)
	at org.apache.iceberg.parquet.ParquetWriter.close(ParquetWriter.java:254)
	at org.apache.iceberg.io.DataWriter.close(DataWriter.java:82)
	at org.apache.iceberg.io.RollingFileWriter.closeCurrentWriter(RollingFileWriter.java:122)
	at org.apache.iceberg.io.RollingFileWriter.close(RollingFileWriter.java:147)
	at org.apache.iceberg.io.RollingDataWriter.close(RollingDataWriter.java:32)
	at org.apache.iceberg.spark.source.SparkWrite$UnpartitionedDataWriter.close(SparkWrite.java:747)
	at org.apache.iceberg.spark.source.SparkWrite$UnpartitionedDataWriter.commit(SparkWrite.java:729)
	at org.apache.spark.sql.execution.datasources.v2.WritingSparkTask.$anonfun$run$5(WriteToDataSourceV2E

Py4JJavaError: An error occurred while calling o247.createOrReplace.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (b26b55345eca executor driver): java.io.UncheckedIOException: Failed to create Parquet file
	at org.apache.iceberg.parquet.ParquetWriter.ensureWriterInitialized(ParquetWriter.java:121)
	at org.apache.iceberg.parquet.ParquetWriter.flushRowGroup(ParquetWriter.java:210)
	at org.apache.iceberg.parquet.ParquetWriter.close(ParquetWriter.java:254)
	at org.apache.iceberg.io.DataWriter.close(DataWriter.java:82)
	at org.apache.iceberg.io.RollingFileWriter.closeCurrentWriter(RollingFileWriter.java:122)
	at org.apache.iceberg.io.RollingFileWriter.close(RollingFileWriter.java:147)
	at org.apache.iceberg.io.RollingDataWriter.close(RollingDataWriter.java:32)
	at org.apache.iceberg.spark.source.SparkWrite$UnpartitionedDataWriter.close(SparkWrite.java:747)
	at org.apache.icebe

In [5]:
spark.sql("select * from iceberg_catalog.iceberg_db.user limit 10").show()

+----------------+--------------------+--------------------+--------------------+
|            name|             address|               email|        phone_number|
+----------------+--------------------+--------------------+--------------------+
|  Sheri Robinson|32541 Petersen Bu...|levinelisa@exampl...|   644-574-1861x9831|
|  Julie Callahan|4766 Lauren Plaza...|michellesalazar@e...|        292-645-8248|
| Jessica Morales|69516 Hughes Viad...|jamesrivers@examp...|    001-730-848-8939|
|     Lisa Torres|175 David Pine\nS...|michelle50@exampl...|001-751-780-8390x...|
|  Scott Williams|6842 Lucas Walk\n...|wmckenzie@example...|        374.942.2125|
|  Ashley Jackson|031 Brown Wells S...|mathewellis@examp...|  445-844-8485x23088|
|  Arthur Griffin|608 Wilson Brooks...|jeffreymoran@exam...| +1-720-277-5627x834|
|Catherine Romero|0788 Contreras Tr...|jordancynthia@exa...|    001-520-762-9716|
|  Matthew Harvey|905 Jones Parkway...|  hstout@example.com| +1-712-465-0887x000|
|     Robin Smit

In [14]:
spark.stop()